# SimplicialCliqueLifting Tutorial

In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)
root_folder = rootutils.find_root()
import omegaconf

from topobenchmarkx.data.load.loaders import GraphLoader

### Load dataset

### Import Dataset Config

Either we keep yaml config files and provide a brief overview of them, or we build the required config files by hand in these tutorials. (I prefer the former option.)

In [2]:
dataset_name = "cocitation_cora"
dataset_config = omegaconf.OmegaConf.load(
    f"{root_folder}/configs/dataset/{dataset_name}.yaml"
).parameters
#################################################################################################################
# Need to interpolate a couple of paths by hand for now; will be solved when we generate the challenge repository:
dataset_config["data_dir"] = (
    f"{root_folder}/datasets/{dataset_config['data_domain']}/{dataset_config['data_type']}"
)
dataset_config["data_split_dir"] = (
    f"{dataset_config['data_dir']}/data_splits/{dataset_config['data_name']}"
)
#################################################################################################################
dataset_config

{'data_domain': 'graph', 'data_type': 'cocitation', 'data_name': 'Cora', 'data_dir': '/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/datasets/graph/cocitation', 'data_split_dir': '/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/datasets/graph/cocitation/data_splits/Cora', 'num_features': 1433, 'num_classes': 7, 'task': 'classification', 'loss_type': 'cross_entropy', 'monitor_metric': 'accuracy', 'task_level': 'node', 'data_seed': 0, 'split_type': 'random', 'k': 10, 'train_prop': 0.5, 'batch_size': 1, 'num_workers': 1, 'pin_memory': False}

### Import Transform Config

Same dilemma as before, yaml files or dicts within tutorials.

In [10]:
lifting_type = "graph2simplicial"
id_lifting = "simplicial_clique"
transform_config = {
    "lifting": omegaconf.OmegaConf.load(
        f"{root_folder}/configs/dataset/transforms/{lifting_type}_lifting/{id_lifting}.yaml"
    )
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}
print(transform_config)

{'lifting': {'_target_': 'topobenchmarkx.transforms.data_transform.DataTransform', 'transform_type': 'lifting', 'transform_name': 'SimplicialCliqueLifting', 'complex_dim': '${oc.select:dataset.parameters.max_dim_if_lifted,2}', 'preserve_edge_attr': '${oc.select:dataset.parameters.preserve_edge_attr_if_lifted,False}', 'signed': True, 'feature_lifting': 'ProjectionSum'}}


### Load and Transform the Dataset

In [13]:
dataset = GraphLoader(dataset_config, transform_config).load()

Transform parameters are the same, using existing data_dir: /Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/datasets/graph/cocitation/Cora/lifting/4278182681


/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:293: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


### Create a Neural Network Model

In [31]:
import torch
from topomodelx.nn.simplicial.san import SAN


class Network(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, n_layers=1):
        super().__init__()
        self.base_model = SAN(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            n_layers=n_layers,
        )
        self.linear = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, laplacian_up, laplacian_down):
        x = self.base_model(x, laplacian_up, laplacian_down)
        x = self.linear(x)
        return torch.sigmoid(x)

### Run the Model

In [36]:
n_layers = 2
in_channels = dataset_config["num_features"]
hidden_channels = 32
out_channels = dataset_config["num_classes"]

model = Network(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,
    n_layers=n_layers,
)

data = dataset.data_lst[0]

In [40]:
y_hat = model(data.x_1, data.up_laplacian_1, data.down_laplacian_1)